In [49]:
import pandas as pd
import numpy as np
import sqlite3
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [44]:
timeline = pd.read_csv('./crypto_reg_timeline.csv')
timeline['Date'] = pd.to_datetime(timeline['Date'], format='%Y%m%d')
timeline

,Date,Country/Region,Topic
0,2018-06-11,UK,AML/CFT
1,2019-10-01,US,Cyrpto firms can become SPDIs
2,2019-12-16,HK,Risk assessment for VASPs and AML/CFT
3,2020-01-30,US,OCC issued cease-and-desist to Safra Bank for ...
4,2020-06-04,US,OCC fair access to banking services
5,2020-06-22,US,OCC banks can provide crypto custodiary services
6,2020-09-16,US,Kraken becomes SPDI
7,2020-09-21,US,OCC banks can hold reserves for stablecoin iss...
8,2021-01-04,US,OCC banks can engage in some payment-activitie...
9,2021-01-11,US,OCC recognizes authority of crypto business as...


In [45]:
tweets = pd.read_csv('Bitcoin_tweets_dataset_2.csv', lineterminator='\n')
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ChefSam,Sunshine State,Culinarian | Hot Sauce Artisan | Kombucha Brew...,2011-03-23 03:50:13,4680,2643,6232,False,2023-03-01 23:59:59,Which #bitcoin books should I think about read...,['bitcoin'],Twitter for iPhone,False
1,Roy⚡️,NaN,Truth-seeking pleb 📚 • Science 🧪 • Nature 🌱☀️ ...,2022-01-30 17:41:41,770,1145,9166,False,2023-03-01 23:59:47,"@ThankGodForBTC I appreciate the message, but ...",['Bitcoin'],Twitter for iPhone,False
2,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,576,1,0,False,2023-03-01 23:59:42,#Ethereum price update: \n\n#ETH $1664.02 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
3,Viction,"Paris, France",https://t.co/8M3rgdjwEe\n\n#bitcoin #blockchai...,2010-03-26 10:15:26,236,1829,2195,False,2023-03-01 23:59:36,CoinDashboard v3.0 is here\nAvailable on ios a...,['Bitcoin'],Twitter for Android,False
4,Rosie,London,"The flower language of jasmine is loyalty, res...",2013-02-16 09:57:56,12731,46,134,False,2023-03-01 23:59:32,#Bitcoin Short Term Fractal (4H)💥\n\nIn lower ...,"['Bitcoin', 'BTC']",Twitter Web App,False


In [46]:
tweets['user_created'] = tweets['user_created'].str[:10]
tweets['user_created'] = pd.to_datetime(tweets['user_created'][:10], format='%Y-%m-%d')

tweets.rename(columns={'user_created': 'Date'}, inplace=True)

merged_df = pd.merge(timeline, tweets, on='Date', how='inner')
merged_df

,Date,Country/Region,Topic,user_name,user_location,user_description,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet


In [50]:
#sentiment analysis for postive or negative
nltk.download('vader_lexicon')

def get_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']
    
tweets['sentiment_score'] = tweets['text'].apply(get_sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/adammualem/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [51]:
tweets

,user_name,user_location,user_description,Date,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,sentiment_score
0,ChefSam,Sunshine State,Culinarian | Hot Sauce Artisan | Kombucha Brew...,2011-03-23,4680,2643,6232,False,2023-03-01 23:59:59,Which #bitcoin books should I think about read...,['bitcoin'],Twitter for iPhone,False,0.0000
1,Roy⚡️,NaN,Truth-seeking pleb 📚 • Science 🧪 • Nature 🌱☀️ ...,2022-01-30,770,1145,9166,False,2023-03-01 23:59:47,"@ThankGodForBTC I appreciate the message, but ...",['Bitcoin'],Twitter for iPhone,False,-0.1513
2,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24,576,1,0,False,2023-03-01 23:59:42,#Ethereum price update: \n\n#ETH $1664.02 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False,0.0000
3,Viction,"Paris, France",https://t.co/8M3rgdjwEe\n\n#bitcoin #blockchai...,2010-03-26,236,1829,2195,False,2023-03-01 23:59:36,CoinDashboard v3.0 is here\nAvailable on ios a...,['Bitcoin'],Twitter for Android,False,0.0000
4,Rosie,London,"The flower language of jasmine is loyalty, res...",2013-02-16,12731,46,134,False,2023-03-01 23:59:32,#Bitcoin Short Term Fractal (4H)💥\n\nIn lower ...,"['Bitcoin', 'BTC']",Twitter Web App,False,-0.2960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169815,GameFiDash | P2E Game | NFT | Web 3.0,NaN,Homepage:https://t.co/tKbZ1UwZ9Q\nTelegram: ht...,NaT,1633,1704,823,False,2023-03-04 10:51:00,#BTC Price $22338\n\n-RSI: 52\n-Moving Average...,['BTC'],Twitter Web App,False,0.0000
169816,Niklas Feurstein,Austria,"Blockchain Advocate👨‍💼, Programmer💻, Crypto En...",NaT,7448,10,7341,False,2023-03-04 10:50:48,Jack Dorsey’s TBD launches ‘C=’ to improve Bit...,"['crypto', 'btc']",RSS Reader - Crypto Posts,False,0.4404
169817,ERROR.404 💱,SUPREME ORDER OF T.,"lil rude, forgive me..\n\n2020 / 2025 / 2028 /...",NaT,421,428,20128,False,2023-03-04 10:50:29,@TheBlock__ Every holder in Celsius fcked 🤓\nT...,['BTC'],Twitter for Android,False,0.0000
169818,AI_Cryptominati/100x-1000x gems,sweden,Low of attraction/ I am a Crypto Millionaire! ...,NaT,672,2031,465,False,2023-03-04 10:50:17,#btc #altcoin #short https://t.co/cAoqF22klA v...,"['btc', 'altcoin', 'short']",Twitter for iPhone,False,0.0000


In [47]:
engine = sqlite3.connect('./btc-data.db')
btc = pd.read_sql('SELECT * FROM BTCUSD', engine)

In [48]:
btc.head()

,date,open,high,low,close,volume,transactions,log,logRet
0,2023-11-22 06:15:00,36475.79,36492.0,36413.6,36443.80,82.165437,1840,10.503527,-0.000877
1,2023-11-22 06:00:00,36371.84,36490.0,36359.6,36475.79,188.767824,2105,10.504404,0.002854
2,2023-11-22 05:45:00,36406.20,36413.0,36341.2,36371.85,60.857387,2005,10.501550,-0.000944
3,2023-11-22 05:30:00,36410.40,36414.0,36355.2,36406.20,208.599722,2383,10.502494,0.000390
4,2023-11-22 05:15:00,36488.10,36488.2,36390.9,36392.00,96.059876,2276,10.502104,-0.001952
